In [ ]:
"""
feature extraction:
Using t1ce, flair and the corresponding computed mask to generate features through pyradiomics
Then, include age and resection status as well 

feature selection:


prediction:
regression model



"""

In [1]:
# Necessary Library Calls
import nibabel as nib
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from skimage.util import montage 
from skimage.transform import rotate
from sklearn.model_selection import train_test_split as ttt
import torch
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm import tqdm
from pylab import *
import gc
import torchmetrics 
import radiomics
from torchvision.utils import save_image
import matplotlib.image
import SimpleITK as sitk
import radiomics



C:\Users\timlu\anaconda3\envs\FYP\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Global variable definition
IMAGE_PATH = 'data/MICCAI_BraTS_2019_Data_Training'
CSV_FILE='data/MICCAI_BraTS_2019_Data_Training/new.csv'
MODEL_PATH = 'log/22/'

# number of slices, meaning how many slices we are taking from one image file  
SLICE_NUM = 10
# Original Image Size
IMAGE_SIZE=240
# Image size after cropping
HEIGHT=128
WIDTH=144

# Batch size constant
BATCH_SIZE= 1
# input_channel corresponds to the number of different types of images used: 
# e.g: T1, T2, T1_flair, etc. 
# only using one type of images at the moment
INPUT_CHANNEL=2
EPOCH = 50
LEARNING_RATE = 0.001
SLICE_STARTS=torch.load('data/slice_starts.pt')
device = torch.device('cuda')
LOAD_CURRENT = False
LOAD_BEST = True


In [3]:
"""

Data loader for easy data access. 

Output shape for get_item: X [slices, input_channels, W, H ]
                           y [slices, W, H, ]
Notes: the output channel for y has dimension of 1 and has value in range of (0,3).


"""


# Simple helper function for retriving images given the path
def get_image(image_path,image_cat,image_id,image_type):
    t1_data=nib.load(os.path.join(image_path,image_cat,image_id,'')+image_id+'_t1.nii.gz').get_fdata()
    t1ce_data=nib.load(os.path.join(image_path,image_cat,image_id,'')+image_id+'_t1ce.nii.gz').get_fdata()
    t2_data=nib.load(os.path.join(image_path,image_cat,image_id,'')+image_id+'_t2.nii.gz').get_fdata()
    seg_data=nib.load(image_path+'/'+image_cat+'/'+image_id+'/'+image_id+'_seg.nii.gz').get_fdata()
    flair_data=nib.load(os.path.join(image_path,image_cat,image_id,'')+image_id+'_flair.nii.gz').get_fdata()
    result={'t1':t1_data,'t1ce':t1ce_data,'t2':t2_data,'seg':seg_data,'flair':flair_data}
   
    return result[image_type]; 


class BraTS19Dataset(Dataset):
    def __init__(self, csv_f,i_path,i_type):
        self.source=pd.read_csv(csv_f)
        self.image_path=i_path
        self.type=i_type
        self.batch=BATCH_SIZE
    def __len__(self):
        return len(self.source)
     
    def __getitem__(self,idx):
        image_cat=self.source.iloc[idx,0]
        image_id=self.source.iloc[idx,1]
        starts=SLICE_STARTS[idx].int()
        age=self.source.iloc[idx,2]
        sur=self.source.iloc[idx,3]
        stu=self.source.iloc[idx,4]
        if sur<300: 
            cla = 0
        elif sur<450:
            cla=1
        else:
            cla =2
        #classification class 
        
        # get the image and its corresponding mask 
        img = np.zeros((INPUT_CHANNEL,IMAGE_SIZE,IMAGE_SIZE,155))
        
        for i in range(INPUT_CHANNEL):
            img[i]=get_image(self.image_path,image_cat,image_id,self.type[i])
        
     
        mask=get_image(self.image_path,image_cat,image_id,'seg')
        
        
        # change label 4 to 3 for easy index handling
        
        mask[mask==4]=3
        
        # initialize results arrays
        X=np.zeros((INPUT_CHANNEL,HEIGHT,WIDTH,SLICE_NUM))
        y=np.zeros((HEIGHT,WIDTH,SLICE_NUM))
        X = img[:,50:50+HEIGHT,55:55+WIDTH,starts-5:starts-5+SLICE_NUM]
        y = mask[50:50+HEIGHT,55:55+WIDTH,starts-5:starts-5+SLICE_NUM]
        X=torch.from_numpy(X)
        y=torch.from_numpy(y)
        
        
      
       
       
        y = y.long()
#        y = F.one_hot(y,num_classes=4)
        return (X.permute(3,0,1,2),y.permute(2,0,1),(age,sur,stu,cla))
    
def GetBraTS19Dataset(csv,ipath,itype,batch_size=1,shuffle=True,pin_memory=True):
    data = BraTS19Dataset(csv,ipath,itype)
    leng = [int(0.6* len(data)),int(0.2* len(data)),len(data) - int(0.6 * len(data))-int(0.2 * len(data))]
    train,eva,test=torch.utils.data.random_split(data,leng)
    train=DataLoader(train,batch_size=batch_size,shuffle=shuffle,pin_memory=pin_memory)
    eva=DataLoader(eva,batch_size=batch_size,shuffle=shuffle,pin_memory=pin_memory)
    test=DataLoader(eva,batch_size=batch_size,shuffle=shuffle,pin_memory=pin_memory)
    return train,eva,test

In [4]:
# Unet model
class Unet(nn.Module):

    def encoder_block(self,in_channels,out_channels,k=3,conv2d_pad1=(1,1),conv2d_pad2=(1,1)):
        block = nn.Sequential(
            nn.Conv2d(in_channels,out_channels,kernel_size=k,padding=conv2d_pad1),
            nn.ReLU(inplace=True),
            nn.GroupNorm(4,out_channels),
            nn.Conv2d(out_channels,out_channels,kernel_size=k,padding=conv2d_pad2),
            nn.ReLU(inplace=True),
            nn.GroupNorm(4,out_channels),
        )
        return block
    
    # actual output channel size = output_channel/2
    def decoder_block(self,in_channels,out_channels,k=3,s=2,conv2d_pad1=(1,1),conv2d_pad2=(1,1),convtrans2d_pad=(0,0)):
        block = nn.Sequential(
            nn.Conv2d(in_channels,out_channels,kernel_size=k,padding=conv2d_pad1),
            nn.GroupNorm(4,out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels,out_channels,kernel_size=k,padding=conv2d_pad2),
            nn.ReLU(inplace=True),
            nn.GroupNorm(4,out_channels),
            nn.ConvTranspose2d(out_channels,int(out_channels/2),kernel_size=2,stride=s,padding=convtrans2d_pad),
        )
        return block
    
    def bottle_neck(self,in_channels,out_channels,k=3):
        block = nn.Sequential(
            nn.Conv2d(in_channels,out_channels,kernel_size=k,stride=1,padding=1),
            nn.GroupNorm(4,out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels,out_channels,kernel_size=k,stride=1,padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.GroupNorm(4,out_channels),
            nn.ConvTranspose2d(out_channels,in_channels,kernel_size=2,stride=2,padding=(0,0),output_padding=(0,0)),
        )
        return block
    
        
    def final_block(self,in_channels,out_channels,k):
        block = nn.Sequential(
            nn.Conv2d(in_channels,int(in_channels/2),kernel_size=k,padding=(1,1)),
            nn.GroupNorm(4,int(in_channels/2)),
            nn.ReLU(inplace=True),
            nn.Conv2d(int(in_channels/2),out_channels,kernel_size=k,padding=(1,1)),
            nn.ReLU(inplace=True),
            nn.GroupNorm(4,out_channels),
            nn.Conv2d(out_channels,out_channels,kernel_size=1,stride=1),
            nn.Softmax(dim=1),
            
        )
        return block
    
    def __init__(self, in_channel, out_channel):
        super(Unet, self).__init__()
        self.en1=self.encoder_block(in_channel,32)
        self.max1 = nn.MaxPool2d(kernel_size=2,stride=2)
        self.en2=self.encoder_block(32,64)
        self.max2 = nn.MaxPool2d(kernel_size=2,stride=2)
        self.en3=self.encoder_block(64,128)
        self.max3 = nn.MaxPool2d(kernel_size=2,stride=2)
        self.en4=self.encoder_block(128,256)
        self.max4 = nn.MaxPool2d(kernel_size=2,stride=2)
        self.bn=self.bottle_neck(256,512)
        self.dc4=self.decoder_block(512,256)
        self.dc3=self.decoder_block(256,128)
        self.dc2=self.decoder_block(128,64)
        self.final=self.final_block(64,out_channel,k=3)
   
    def forward(self,x):
        encoder_block1=self.en1(x)
        mp1 = self.max1(encoder_block1)
        encoder_block2=self.en2(mp1)
        mp2 = self.max2(encoder_block2)
        encoder_block3=self.en3(mp2)
        mp3 = self.max3(encoder_block3)
        encoder_block4=self.en4(mp3)
        mp4 = self.max4(encoder_block4)
        bottleneck_block1 =self.bn(mp4)
        
        cat_block4= torch.cat((bottleneck_block1,encoder_block4),1)
        decoder_block4=self.dc4(cat_block4)
        
        cat_block3= torch.cat((decoder_block4,encoder_block3),1)
        decoder_block3=self.dc3(cat_block3)
        
        cat_block2= torch.cat((decoder_block3,encoder_block2),1)
        decoder_block2=self.dc2(cat_block2)
        
        cat_block1= torch.cat((decoder_block2,encoder_block1),1)
        
        result=self.final(cat_block1)
        
        return result

In [5]:
"""
x shape: [slice_num = 10 ,input_channel = 2, h, w]
y shape: [slice_num = 10, h, w]
res shape (after argmax): [slice_num = 10, h, w]


3d features: 16 * 2
first order features: 19 * 2

Cluster Shade, Correlation, Small Dependence High Gray Level Emphasis
Cluster Prominence, Gray Level Non-Uniformity (GLN),Gray Level Variance (GLV)
Large dependence low gray level emphasis, Large Area Emphasis (LAE)

Age, resection status

"""



def feature_extraction():
    # Load data
    train_data = torch.load(MODEL_PATH+'trdata.pth')
    eval_data = torch.load(MODEL_PATH+'edata.pth')
    test_data = torch.load(MODEL_PATH+'tdata.pth')
    
    tlen = len(train_data.sampler)
    elen= len(eval_data.sampler)
    ttlen=len(test_data.sampler)
    
    # features from the train data set
    train_feature=torch.zeros((tlen,88))
    # features from the eval data set
    eval_feature=torch.zeros((elen,88))
    # features from the eval data set
    test_feature=torch.zeros((ttlen,88))
    
    # survival days of the patient
    train_surv=torch.zeros((tlen))
    # survival days of the patient
    eval_surv=torch.zeros((elen))
    # survival days of the patient
    test_surv=torch.zeros((ttlen))
    
    
    
    checkpoint = torch.load(MODEL_PATH+'#22-BEST')
    model = Unet(in_channel=INPUT_CHANNEL,out_channel=4).to(device)
    opt = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    model.load_state_dict(checkpoint['model_state_dict'])
    opt.load_state_dict(checkpoint['optim_state_dict'])
    
    print('Best Model loaded!')
 


    #training data set
    print('Training data feature extraction: ')
    model.eval()
    
    """

    train data

    """  
    train_data=tqdm(train_data)
    with torch.no_grad():
        
        for index, batch in enumerate(train_data):
            features = torch.zeros(88)
            x, y,other= batch 
            a,b,c,d,e=x.shape
            x=x.reshape(a*b,c,d,e)
            x=x.float()
        
            a,b,c,d=y.shape
            y=y.reshape(a*b,c,d)
            
            age,sur,status,cla = other
            x,y=x.to(device),y.to(device)
            res = model(x)
            res=res.to(device)
            res = torch.argmax(res,dim=1)
            
            # survival info 
            train_surv[index]=sur
            
            im1 = x[:,0,:,:]
            im2 = x[:,1,:,:]
            
            t1ce = sitk.GetImageFromArray(im1.cpu())
            flair = sitk.GetImageFromArray(im2.cpu())
            
            # seg1: computed mask;
            # seg2: ground truth mask;
            seg1 = sitk.GetImageFromArray(res.cpu())
            seg2 = sitk.GetImageFromArray(y.cpu())
            
            """
            computed mask features: 
            
            """
            # t1ce computed mask shape features 
            shape=radiomics.shape.RadiomicsShape(t1ce,seg1)
            
            features[0]=shape.getMeshVolumeFeatureValue()
            features[1]=shape.getVoxelVolumeFeatureValue()
            features[2]=shape.getSurfaceAreaFeatureValue()
            features[3]=shape.getSphericityFeatureValue()
            features[4]=shape.getCompactness1FeatureValue()
            features[5]=shape.getCompactness2FeatureValue()
            features[6]=shape.getSphericalDisproportionFeatureValue()
            features[7]=shape.getMaximum3DDiameterFeatureValue()
            features[8]=shape.getMaximum2DDiameterSliceFeatureValue()
            features[9]=shape.getMaximum2DDiameterColumnFeatureValue()
            features[10]=shape.getMaximum2DDiameterRowFeatureValue()
            features[11]=shape.getMajorAxisLengthFeatureValue()
            features[12]=shape.getMinorAxisLengthFeatureValue()
            features[13]=shape.getLeastAxisLengthFeatureValue()
            features[14]=shape.getElongationFeatureValue()
            features[15]=shape.getFlatnessFeatureValue()
            
            # flair computed mask shape features 
            shape=radiomics.shape.RadiomicsShape(flair,seg1)
            
            features[16]=shape.getMeshVolumeFeatureValue()
            features[17]=shape.getVoxelVolumeFeatureValue()
            features[18]=shape.getSurfaceAreaFeatureValue()
            features[19]=shape.getSphericityFeatureValue()
            features[20]=shape.getCompactness1FeatureValue()
            features[21]=shape.getCompactness2FeatureValue()
            features[22]=shape.getSphericalDisproportionFeatureValue()
            features[23]=shape.getMaximum3DDiameterFeatureValue()
            features[24]=shape.getMaximum2DDiameterSliceFeatureValue()
            features[25]=shape.getMaximum2DDiameterColumnFeatureValue()
            features[26]=shape.getMaximum2DDiameterRowFeatureValue()
            features[27]=shape.getMajorAxisLengthFeatureValue()
            features[28]=shape.getMinorAxisLengthFeatureValue()
            features[29]=shape.getLeastAxisLengthFeatureValue()
            features[30]=shape.getElongationFeatureValue()
            features[31]=shape.getFlatnessFeatureValue()
            
            # t1ce and computed mask first order features
            fo = radiomics.firstorder.RadiomicsFirstOrder(t1ce,seg1)
            fo._initCalculation()

            features[32]=(torch.tensor(fo.getEnergyFeatureValue()))
            features[33]=(torch.tensor(fo.getTotalEnergyFeatureValue()))
            features[34]=(torch.tensor(fo.getEntropyFeatureValue()))
            features[35]=(torch.tensor(fo.getMinimumFeatureValue()))
            features[36]=(torch.tensor(fo.get10PercentileFeatureValue()))
            features[37]=(torch.tensor(fo.get90PercentileFeatureValue()))
            features[38]=(torch.tensor(fo.getMaximumFeatureValue()))
            features[39]=(torch.tensor(fo.getMeanFeatureValue()))
            features[40]=(torch.tensor(fo.getMedianFeatureValue()))
            features[41]=(torch.tensor(fo.getInterquartileRangeFeatureValue()))
            features[42]=(torch.tensor(fo.getRangeFeatureValue()))
            features[43]=(torch.tensor(fo.getMeanAbsoluteDeviationFeatureValue()))
            features[44]=(torch.tensor(fo.getRobustMeanAbsoluteDeviationFeatureValue()))
            features[45]=(torch.tensor(fo.getRootMeanSquaredFeatureValue()))
            features[46]=(torch.tensor(fo.getStandardDeviationFeatureValue()))
            features[47]=(torch.tensor(fo.getSkewnessFeatureValue()))
            features[48]=(torch.tensor(fo.getKurtosisFeatureValue()))
            features[49]=(torch.tensor(fo.getVarianceFeatureValue()))
            features[50]=(torch.tensor(fo.getUniformityFeatureValue()))
            
            # flair and computed mask first order features
            fo = radiomics.firstorder.RadiomicsFirstOrder(flair,seg1)
            fo._initCalculation()
            
            features[51]=(torch.tensor(fo.getEnergyFeatureValue()))
            features[52]=(torch.tensor(fo.getTotalEnergyFeatureValue()))
            features[53]=(torch.tensor(fo.getEntropyFeatureValue()))
            features[54]=(torch.tensor(fo.getMinimumFeatureValue()))
            features[55]=(torch.tensor(fo.get10PercentileFeatureValue()))
            features[56]=(torch.tensor(fo.get90PercentileFeatureValue()))
            features[57]=(torch.tensor(fo.getMaximumFeatureValue()))
            features[58]=(torch.tensor(fo.getMeanFeatureValue()))
            features[59]=(torch.tensor(fo.getMedianFeatureValue()))
            features[60]=(torch.tensor(fo.getInterquartileRangeFeatureValue()))
            features[61]=(torch.tensor(fo.getRangeFeatureValue()))
            features[62]=(torch.tensor(fo.getMeanAbsoluteDeviationFeatureValue()))
            features[63]=(torch.tensor(fo.getRobustMeanAbsoluteDeviationFeatureValue()))
            features[64]=(torch.tensor(fo.getRootMeanSquaredFeatureValue()))
            features[65]=(torch.tensor(fo.getStandardDeviationFeatureValue()))
            features[66]=(torch.tensor(fo.getSkewnessFeatureValue()))
            features[67]=(torch.tensor(fo.getKurtosisFeatureValue()))
            features[68]=(torch.tensor(fo.getVarianceFeatureValue()))
            features[69]=(torch.tensor(fo.getUniformityFeatureValue()))
            
            # t1ce and computed mask grey-level features
            grey = radiomics.glcm.RadiomicsGLCM(t1ce,seg1)
            grey._initCalculation()
            
            features[70]=(torch.tensor(grey.getClusterProminenceFeatureValue()))
            features[71]=(torch.tensor(grey.getClusterShadeFeatureValue()))
            features[72]=(torch.tensor(grey.getCorrelationFeatureValue()))
            
            # flair and computed mask grey-level features
            grey = radiomics.glcm.RadiomicsGLCM(flair,seg1)
            grey._initCalculation()
            
            features[73]=(torch.tensor(grey.getClusterProminenceFeatureValue()))
            features[74]=(torch.tensor(grey.getClusterShadeFeatureValue()))
            features[75]=(torch.tensor(grey.getCorrelationFeatureValue()))
            
            #t1ce grey-level zone features
            grey =radiomics.glszm.RadiomicsGLSZM(t1ce,seg1)
            grey._initCalculation()
            
            features[76]=(torch.tensor(grey.getLargeAreaEmphasisFeatureValue()))
            features[77]=(torch.tensor(grey.getGrayLevelNonUniformityFeatureValue()))
            features[78]=(torch.tensor(grey.getGrayLevelVarianceFeatureValue()))
            
            #flair grey-level zone features
            grey =radiomics.glszm.RadiomicsGLSZM(flair,seg1)
            grey._initCalculation()
            
            features[79]=(torch.tensor(grey.getLargeAreaEmphasisFeatureValue()))
            features[80]=(torch.tensor(grey.getGrayLevelNonUniformityFeatureValue()))
            features[81]=(torch.tensor(grey.getGrayLevelVarianceFeatureValue()))
            
            #t1ce grey-level dependence features
            grey =radiomics.gldm.RadiomicsGLDM(t1ce,seg1)
            grey._initCalculation()
            
            features[82]=(torch.tensor(grey.getSmallDependenceHighGrayLevelEmphasisFeatureValue()))
            features[83]=(torch.tensor(grey.getLargeDependenceLowGrayLevelEmphasisFeatureValue()))
            
            #t1ce grey-level dependence features
            grey =radiomics.gldm.RadiomicsGLDM(flair,seg1)
            grey._initCalculation()
            
            features[84]=(torch.tensor(grey.getSmallDependenceHighGrayLevelEmphasisFeatureValue()))
            features[85]=(torch.tensor(grey.getLargeDependenceLowGrayLevelEmphasisFeatureValue()))
            
            # clinical info
            if status == ['1']:
                features[86]=1
            else:
                features[86]=0
            features[87]=age
            train_feature[index]=features
    """

    eval data

    """  
            
    eval_data=tqdm(eval_data)
    with torch.no_grad():
        
        for index, batch in enumerate(eval_data):
            features = torch.zeros(88)
            x, y,other= batch 
            a,b,c,d,e=x.shape
            x=x.reshape(a*b,c,d,e)
            x=x.float()
        
            a,b,c,d=y.shape
            y=y.reshape(a*b,c,d)
            
            age,sur,status,cla = other
            x,y=x.to(device),y.to(device)
            res = model(x)
            res=res.to(device)
            res = torch.argmax(res,dim=1)
            
            eval_surv[index]=sur
            im1 = x[:,0,:,:]
            im2 = x[:,1,:,:]
            
            t1ce = sitk.GetImageFromArray(im1.cpu())
            flair = sitk.GetImageFromArray(im2.cpu())
            
            # seg1: computed mask;
            # seg2: ground truth mask;
            seg1 = sitk.GetImageFromArray(res.cpu())
            seg2 = sitk.GetImageFromArray(y.cpu())
            
            """
            computed mask features: 
            
            """
            # t1ce computed mask shape features 
            shape=radiomics.shape.RadiomicsShape(t1ce,seg1)
            
            features[0]=shape.getMeshVolumeFeatureValue()
            features[1]=shape.getVoxelVolumeFeatureValue()
            features[2]=shape.getSurfaceAreaFeatureValue()
            features[3]=shape.getSphericityFeatureValue()
            features[4]=shape.getCompactness1FeatureValue()
            features[5]=shape.getCompactness2FeatureValue()
            features[6]=shape.getSphericalDisproportionFeatureValue()
            features[7]=shape.getMaximum3DDiameterFeatureValue()
            features[8]=shape.getMaximum2DDiameterSliceFeatureValue()
            features[9]=shape.getMaximum2DDiameterColumnFeatureValue()
            features[10]=shape.getMaximum2DDiameterRowFeatureValue()
            features[11]=shape.getMajorAxisLengthFeatureValue()
            features[12]=shape.getMinorAxisLengthFeatureValue()
            features[13]=shape.getLeastAxisLengthFeatureValue()
            features[14]=shape.getElongationFeatureValue()
            features[15]=shape.getFlatnessFeatureValue()
            
            # flair computed mask shape features 
            shape=radiomics.shape.RadiomicsShape(flair,seg1)
            
            features[16]=shape.getMeshVolumeFeatureValue()
            features[17]=shape.getVoxelVolumeFeatureValue()
            features[18]=shape.getSurfaceAreaFeatureValue()
            features[19]=shape.getSphericityFeatureValue()
            features[20]=shape.getCompactness1FeatureValue()
            features[21]=shape.getCompactness2FeatureValue()
            features[22]=shape.getSphericalDisproportionFeatureValue()
            features[23]=shape.getMaximum3DDiameterFeatureValue()
            features[24]=shape.getMaximum2DDiameterSliceFeatureValue()
            features[25]=shape.getMaximum2DDiameterColumnFeatureValue()
            features[26]=shape.getMaximum2DDiameterRowFeatureValue()
            features[27]=shape.getMajorAxisLengthFeatureValue()
            features[28]=shape.getMinorAxisLengthFeatureValue()
            features[29]=shape.getLeastAxisLengthFeatureValue()
            features[30]=shape.getElongationFeatureValue()
            features[31]=shape.getFlatnessFeatureValue()
            
            # t1ce and computed mask first order features
            fo = radiomics.firstorder.RadiomicsFirstOrder(t1ce,seg1)
            fo._initCalculation()

            features[32]=(torch.tensor(fo.getEnergyFeatureValue()))
            features[33]=(torch.tensor(fo.getTotalEnergyFeatureValue()))
            features[34]=(torch.tensor(fo.getEntropyFeatureValue()))
            features[35]=(torch.tensor(fo.getMinimumFeatureValue()))
            features[36]=(torch.tensor(fo.get10PercentileFeatureValue()))
            features[37]=(torch.tensor(fo.get90PercentileFeatureValue()))
            features[38]=(torch.tensor(fo.getMaximumFeatureValue()))
            features[39]=(torch.tensor(fo.getMeanFeatureValue()))
            features[40]=(torch.tensor(fo.getMedianFeatureValue()))
            features[41]=(torch.tensor(fo.getInterquartileRangeFeatureValue()))
            features[42]=(torch.tensor(fo.getRangeFeatureValue()))
            features[43]=(torch.tensor(fo.getMeanAbsoluteDeviationFeatureValue()))
            features[44]=(torch.tensor(fo.getRobustMeanAbsoluteDeviationFeatureValue()))
            features[45]=(torch.tensor(fo.getRootMeanSquaredFeatureValue()))
            features[46]=(torch.tensor(fo.getStandardDeviationFeatureValue()))
            features[47]=(torch.tensor(fo.getSkewnessFeatureValue()))
            features[48]=(torch.tensor(fo.getKurtosisFeatureValue()))
            features[49]=(torch.tensor(fo.getVarianceFeatureValue()))
            features[50]=(torch.tensor(fo.getUniformityFeatureValue()))
            
            # flair and computed mask first order features
            fo = radiomics.firstorder.RadiomicsFirstOrder(flair,seg1)
            fo._initCalculation()
            
            features[51]=(torch.tensor(fo.getEnergyFeatureValue()))
            features[52]=(torch.tensor(fo.getTotalEnergyFeatureValue()))
            features[53]=(torch.tensor(fo.getEntropyFeatureValue()))
            features[54]=(torch.tensor(fo.getMinimumFeatureValue()))
            features[55]=(torch.tensor(fo.get10PercentileFeatureValue()))
            features[56]=(torch.tensor(fo.get90PercentileFeatureValue()))
            features[57]=(torch.tensor(fo.getMaximumFeatureValue()))
            features[58]=(torch.tensor(fo.getMeanFeatureValue()))
            features[59]=(torch.tensor(fo.getMedianFeatureValue()))
            features[60]=(torch.tensor(fo.getInterquartileRangeFeatureValue()))
            features[61]=(torch.tensor(fo.getRangeFeatureValue()))
            features[62]=(torch.tensor(fo.getMeanAbsoluteDeviationFeatureValue()))
            features[63]=(torch.tensor(fo.getRobustMeanAbsoluteDeviationFeatureValue()))
            features[64]=(torch.tensor(fo.getRootMeanSquaredFeatureValue()))
            features[65]=(torch.tensor(fo.getStandardDeviationFeatureValue()))
            features[66]=(torch.tensor(fo.getSkewnessFeatureValue()))
            features[67]=(torch.tensor(fo.getKurtosisFeatureValue()))
            features[68]=(torch.tensor(fo.getVarianceFeatureValue()))
            features[69]=(torch.tensor(fo.getUniformityFeatureValue()))
            
            # t1ce and computed mask grey-level features
            grey = radiomics.glcm.RadiomicsGLCM(t1ce,seg1)
            grey._initCalculation()
            
            features[70]=(torch.tensor(grey.getClusterProminenceFeatureValue()))
            features[71]=(torch.tensor(grey.getClusterShadeFeatureValue()))
            features[72]=(torch.tensor(grey.getCorrelationFeatureValue()))
            
            # flair and computed mask grey-level features
            grey = radiomics.glcm.RadiomicsGLCM(flair,seg1)
            grey._initCalculation()
            
            features[73]=(torch.tensor(grey.getClusterProminenceFeatureValue()))
            features[74]=(torch.tensor(grey.getClusterShadeFeatureValue()))
            features[75]=(torch.tensor(grey.getCorrelationFeatureValue()))
            
            #t1ce grey-level zone features
            grey =radiomics.glszm.RadiomicsGLSZM(t1ce,seg1)
            grey._initCalculation()
            
            features[76]=(torch.tensor(grey.getLargeAreaEmphasisFeatureValue()))
            features[77]=(torch.tensor(grey.getGrayLevelNonUniformityFeatureValue()))
            features[78]=(torch.tensor(grey.getGrayLevelVarianceFeatureValue()))
            
            #flair grey-level zone features
            grey =radiomics.glszm.RadiomicsGLSZM(flair,seg1)
            grey._initCalculation()
            
            features[79]=(torch.tensor(grey.getLargeAreaEmphasisFeatureValue()))
            features[80]=(torch.tensor(grey.getGrayLevelNonUniformityFeatureValue()))
            features[81]=(torch.tensor(grey.getGrayLevelVarianceFeatureValue()))
            
            #t1ce grey-level dependence features
            grey =radiomics.gldm.RadiomicsGLDM(t1ce,seg1)
            grey._initCalculation()
            
            features[82]=(torch.tensor(grey.getSmallDependenceHighGrayLevelEmphasisFeatureValue()))
            features[83]=(torch.tensor(grey.getLargeDependenceLowGrayLevelEmphasisFeatureValue()))
            
            #t1ce grey-level dependence features
            grey =radiomics.gldm.RadiomicsGLDM(flair,seg1)
            grey._initCalculation()
            
            features[84]=(torch.tensor(grey.getSmallDependenceHighGrayLevelEmphasisFeatureValue()))
            features[85]=(torch.tensor(grey.getLargeDependenceLowGrayLevelEmphasisFeatureValue()))
            
            # clinical info
            if status == ['1']:
                features[86]=1
            else:
                features[86]=0
            features[87]=age
            eval_feature[index]=features
    """

    test data

    """     
    test_data=tqdm(test_data.dataset)
    with torch.no_grad():
        
        for index, batch in enumerate(test_data):
            features = torch.zeros(88)
            x, y,other= batch 
            a,b,c,d,e=x.shape
            x=x.reshape(a*b,c,d,e)
            x=x.float()
        
            a,b,c,d=y.shape
            y=y.reshape(a*b,c,d)
            
            age,sur,status,cla = other
            x,y=x.to(device),y.to(device)
            res = model(x)
            res=res.to(device)
            res = torch.argmax(res,dim=1)
            
            test_surv[index]=sur
            im1 = x[:,0,:,:]
            im2 = x[:,1,:,:]
            
            t1ce = sitk.GetImageFromArray(im1.cpu())
            flair = sitk.GetImageFromArray(im2.cpu())
            
            # seg1: computed mask;
            # seg2: ground truth mask;
            seg1 = sitk.GetImageFromArray(res.cpu())
            seg2 = sitk.GetImageFromArray(y.cpu())
            
            """
            computed mask features: 
            
            """
            # t1ce computed mask shape features 
            shape=radiomics.shape.RadiomicsShape(t1ce,seg1)
            
            features[0]=shape.getMeshVolumeFeatureValue()
            features[1]=shape.getVoxelVolumeFeatureValue()
            features[2]=shape.getSurfaceAreaFeatureValue()
            features[3]=shape.getSphericityFeatureValue()
            features[4]=shape.getCompactness1FeatureValue()
            features[5]=shape.getCompactness2FeatureValue()
            features[6]=shape.getSphericalDisproportionFeatureValue()
            features[7]=shape.getMaximum3DDiameterFeatureValue()
            features[8]=shape.getMaximum2DDiameterSliceFeatureValue()
            features[9]=shape.getMaximum2DDiameterColumnFeatureValue()
            features[10]=shape.getMaximum2DDiameterRowFeatureValue()
            features[11]=shape.getMajorAxisLengthFeatureValue()
            features[12]=shape.getMinorAxisLengthFeatureValue()
            features[13]=shape.getLeastAxisLengthFeatureValue()
            features[14]=shape.getElongationFeatureValue()
            features[15]=shape.getFlatnessFeatureValue()
            
            # flair computed mask shape features 
            shape=radiomics.shape.RadiomicsShape(flair,seg1)
            
            features[16]=shape.getMeshVolumeFeatureValue()
            features[17]=shape.getVoxelVolumeFeatureValue()
            features[18]=shape.getSurfaceAreaFeatureValue()
            features[19]=shape.getSphericityFeatureValue()
            features[20]=shape.getCompactness1FeatureValue()
            features[21]=shape.getCompactness2FeatureValue()
            features[22]=shape.getSphericalDisproportionFeatureValue()
            features[23]=shape.getMaximum3DDiameterFeatureValue()
            features[24]=shape.getMaximum2DDiameterSliceFeatureValue()
            features[25]=shape.getMaximum2DDiameterColumnFeatureValue()
            features[26]=shape.getMaximum2DDiameterRowFeatureValue()
            features[27]=shape.getMajorAxisLengthFeatureValue()
            features[28]=shape.getMinorAxisLengthFeatureValue()
            features[29]=shape.getLeastAxisLengthFeatureValue()
            features[30]=shape.getElongationFeatureValue()
            features[31]=shape.getFlatnessFeatureValue()
            
            # t1ce and computed mask first order features
            fo = radiomics.firstorder.RadiomicsFirstOrder(t1ce,seg1)
            fo._initCalculation()

            features[32]=(torch.tensor(fo.getEnergyFeatureValue()))
            features[33]=(torch.tensor(fo.getTotalEnergyFeatureValue()))
            features[34]=(torch.tensor(fo.getEntropyFeatureValue()))
            features[35]=(torch.tensor(fo.getMinimumFeatureValue()))
            features[36]=(torch.tensor(fo.get10PercentileFeatureValue()))
            features[37]=(torch.tensor(fo.get90PercentileFeatureValue()))
            features[38]=(torch.tensor(fo.getMaximumFeatureValue()))
            features[39]=(torch.tensor(fo.getMeanFeatureValue()))
            features[40]=(torch.tensor(fo.getMedianFeatureValue()))
            features[41]=(torch.tensor(fo.getInterquartileRangeFeatureValue()))
            features[42]=(torch.tensor(fo.getRangeFeatureValue()))
            features[43]=(torch.tensor(fo.getMeanAbsoluteDeviationFeatureValue()))
            features[44]=(torch.tensor(fo.getRobustMeanAbsoluteDeviationFeatureValue()))
            features[45]=(torch.tensor(fo.getRootMeanSquaredFeatureValue()))
            features[46]=(torch.tensor(fo.getStandardDeviationFeatureValue()))
            features[47]=(torch.tensor(fo.getSkewnessFeatureValue()))
            features[48]=(torch.tensor(fo.getKurtosisFeatureValue()))
            features[49]=(torch.tensor(fo.getVarianceFeatureValue()))
            features[50]=(torch.tensor(fo.getUniformityFeatureValue()))
            
            # flair and computed mask first order features
            fo = radiomics.firstorder.RadiomicsFirstOrder(flair,seg1)
            fo._initCalculation()
            
            features[51]=(torch.tensor(fo.getEnergyFeatureValue()))
            features[52]=(torch.tensor(fo.getTotalEnergyFeatureValue()))
            features[53]=(torch.tensor(fo.getEntropyFeatureValue()))
            features[54]=(torch.tensor(fo.getMinimumFeatureValue()))
            features[55]=(torch.tensor(fo.get10PercentileFeatureValue()))
            features[56]=(torch.tensor(fo.get90PercentileFeatureValue()))
            features[57]=(torch.tensor(fo.getMaximumFeatureValue()))
            features[58]=(torch.tensor(fo.getMeanFeatureValue()))
            features[59]=(torch.tensor(fo.getMedianFeatureValue()))
            features[60]=(torch.tensor(fo.getInterquartileRangeFeatureValue()))
            features[61]=(torch.tensor(fo.getRangeFeatureValue()))
            features[62]=(torch.tensor(fo.getMeanAbsoluteDeviationFeatureValue()))
            features[63]=(torch.tensor(fo.getRobustMeanAbsoluteDeviationFeatureValue()))
            features[64]=(torch.tensor(fo.getRootMeanSquaredFeatureValue()))
            features[65]=(torch.tensor(fo.getStandardDeviationFeatureValue()))
            features[66]=(torch.tensor(fo.getSkewnessFeatureValue()))
            features[67]=(torch.tensor(fo.getKurtosisFeatureValue()))
            features[68]=(torch.tensor(fo.getVarianceFeatureValue()))
            features[69]=(torch.tensor(fo.getUniformityFeatureValue()))
            
            # t1ce and computed mask grey-level features
            grey = radiomics.glcm.RadiomicsGLCM(t1ce,seg1)
            grey._initCalculation()
            
            features[70]=(torch.tensor(grey.getClusterProminenceFeatureValue()))
            features[71]=(torch.tensor(grey.getClusterShadeFeatureValue()))
            features[72]=(torch.tensor(grey.getCorrelationFeatureValue()))
            
            # flair and computed mask grey-level features
            grey = radiomics.glcm.RadiomicsGLCM(flair,seg1)
            grey._initCalculation()
            
            features[73]=(torch.tensor(grey.getClusterProminenceFeatureValue()))
            features[74]=(torch.tensor(grey.getClusterShadeFeatureValue()))
            features[75]=(torch.tensor(grey.getCorrelationFeatureValue()))
            
            #t1ce grey-level zone features
            grey =radiomics.glszm.RadiomicsGLSZM(t1ce,seg1)
            grey._initCalculation()
            
            features[76]=(torch.tensor(grey.getLargeAreaEmphasisFeatureValue()))
            features[77]=(torch.tensor(grey.getGrayLevelNonUniformityFeatureValue()))
            features[78]=(torch.tensor(grey.getGrayLevelVarianceFeatureValue()))
            
            #flair grey-level zone features
            grey =radiomics.glszm.RadiomicsGLSZM(flair,seg1)
            grey._initCalculation()
            
            features[79]=(torch.tensor(grey.getLargeAreaEmphasisFeatureValue()))
            features[80]=(torch.tensor(grey.getGrayLevelNonUniformityFeatureValue()))
            features[81]=(torch.tensor(grey.getGrayLevelVarianceFeatureValue()))
            
            #t1ce grey-level dependence features
            grey =radiomics.gldm.RadiomicsGLDM(t1ce,seg1)
            grey._initCalculation()
            
            features[82]=(torch.tensor(grey.getSmallDependenceHighGrayLevelEmphasisFeatureValue()))
            features[83]=(torch.tensor(grey.getLargeDependenceLowGrayLevelEmphasisFeatureValue()))
            
            #t1ce grey-level dependence features
            grey =radiomics.gldm.RadiomicsGLDM(flair,seg1)
            grey._initCalculation()
            
            features[84]=(torch.tensor(grey.getSmallDependenceHighGrayLevelEmphasisFeatureValue()))
            features[85]=(torch.tensor(grey.getLargeDependenceLowGrayLevelEmphasisFeatureValue()))
            
            # clinical info
            if status == ['1']:
                features[86]=1
            else:
                features[86]=0
            features[87]=age
            test_feature[index]=features    
    
    
    
    

            
    return train_feature,train_surv,eval_feature,eval_surv,test_feature,test_surv
    
            
            
            
  

In [7]:
tf,ts,ef,es,ttf,tts=feature_extraction()

Best Model loaded!
Training data feature extraction: 


100%|██████████| 36/36 [01:01<00:00,  1.71s/it]


In [9]:
FEAT_PATH = "log/feature_extraction/"
tf=torch.nan_to_num(tf)
ef=torch.nan_to_num(ef)
ttf=torch.nan_to_num(ttf)
tc=torch.zeros(len(ts))
ec=torch.zeros(len(ef))
ttc=torch.zeros(len(ttf))
for i in range(len(ts)):
    if ts[i]<300:
        tc[i]=0
    elif ts[i]<450:
        tc[i]=1
    else:
        tc[i]=2
    
for i in range(len(es)):
    if es[i]<300:
        ec[i]=0
    elif es[i]<450:
        ec[i]=1
    else:
        ec[i]=2
for i in range(len(tts)):
    if tts[i]<300:
        ttc[i]=0
    elif tts[i]<450:
        ttc[i]=1
    else:
        ttc[i]=2
        
torch.save(tf,FEAT_PATH+'tf.pt')
torch.save(ef,FEAT_PATH+'ef.pt')
torch.save(ttf,FEAT_PATH+'ttf.pt')

torch.save(ts,FEAT_PATH+'ts.pt')
torch.save(es,FEAT_PATH+'es.pt')
torch.save(tts,FEAT_PATH+'tts.pt')

torch.save(tc,FEAT_PATH+'tc.pt')
torch.save(ec,FEAT_PATH+'ec.pt')
torch.save(ttc,FEAT_PATH+'ttc.pt')

In [10]:
print(tf.shape,ef.shape,ttf.shape)
print(tc.shape,ec.shape,ttc.shape)
print(tc,ec,ttc)

torch.Size([110, 88]) torch.Size([36, 88]) torch.Size([36, 88])
torch.Size([110]) torch.Size([36]) torch.Size([36])
tensor([1., 1., 0., 1., 2., 1., 1., 2., 0., 1., 2., 1., 0., 0., 1., 0., 1., 2.,
        1., 2., 0., 0., 1., 0., 1., 1., 2., 1., 2., 0., 2., 1., 1., 2., 1., 1.,
        1., 0., 1., 0., 1., 0., 2., 1., 2., 1., 1., 0., 1., 1., 0., 0., 2., 1.,
        0., 1., 2., 1., 1., 1., 0., 1., 0., 0., 0., 0., 2., 2., 2., 1., 2., 1.,
        1., 1., 2., 1., 1., 1., 2., 0., 1., 1., 1., 1., 2., 1., 0., 1., 0., 0.,
        0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 2., 1., 1.,
        1., 2.]) tensor([0., 1., 1., 1., 1., 1., 1., 0., 2., 1., 2., 0., 1., 0., 0., 0., 2., 2.,
        1., 1., 1., 0., 1., 2., 2., 0., 0., 2., 2., 2., 0., 1., 1., 1., 1., 1.]) tensor([2., 1., 2., 1., 1., 1., 0., 2., 1., 1., 1., 0., 2., 0., 0., 2., 1., 1.,
        0., 1., 1., 2., 1., 1., 2., 1., 0., 1., 0., 0., 1., 0., 2., 0., 2., 1.])


In [ ]:
"""

Some testing code

"""

In [3]:
FEAT_PATH ='log/feature_extraction/'
tf=torch.load(FEAT_PATH+'tf.pt')
ef=torch.load(FEAT_PATH+'ef.pt')
ttf=torch.load(FEAT_PATH+'ttf.pt')

ts=torch.load(FEAT_PATH+'ts.pt')
es=torch.load(FEAT_PATH+'es.pt')
tts=torch.load(FEAT_PATH+'tts.pt')

tc=torch.load(FEAT_PATH+'tc.pt')
ec=torch.load(FEAT_PATH+'ec.pt')
ttc=torch.load(FEAT_PATH+'ttc.pt')





In [4]:
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression


In [187]:

t1=tf
e1=ef
tt1=ttf
selector = SelectFromModel(estimator=LogisticRegression()).fit(t1, tc)
t1=selector.transform(t1)
e1=selector.transform(e1)
tt1=selector.transform(tt1)

clf=LogisticRegression().fit(t1,tc)
clf.score(e1,ttc)



0.5277777777777778

In [16]:
from sklearn.svm import LinearSVC
t1=tf
e1=ef

tt1=ttf
s = SelectFromModel(estimator=LinearSVC(dual=False)).fit(t1, tc)
t1=s.transform(t1)
e1=s.transform(e1)
tt1=s.transform(tt1)

clf=LinearSVC(dual=False).fit(t1,tc)
clf.score(t1,tc)



0.43636363636363634

In [149]:
selector.get_support()
selector.transform(t1)


array([[1.21208627e+09, 1.21208627e+09, 1.06051859e+09, 1.06051859e+09],
       [1.91398624e+08, 1.91398624e+08, 4.07754848e+08, 4.07754848e+08],
       [6.83496653e+10, 6.83496653e+10, 1.12211354e+10, 1.12211354e+10],
       [3.34279620e+07, 3.34279620e+07, 7.91431840e+07, 7.91431840e+07],
       [2.23592256e+08, 2.23592256e+08, 4.01594752e+08, 4.01594752e+08],
       [1.91421152e+08, 1.91421152e+08, 3.61693824e+08, 3.61693824e+08],
       [5.75114304e+08, 5.75114304e+08, 2.64149376e+08, 2.64149376e+08],
       [3.90097600e+07, 3.90097600e+07, 1.58938800e+07, 1.58938800e+07],
       [3.46162509e+09, 3.46162509e+09, 2.66582963e+09, 2.66582963e+09],
       [3.80605760e+08, 3.80605760e+08, 2.57065024e+08, 2.57065024e+08],
       [6.14488384e+08, 6.14488384e+08, 5.87968832e+08, 5.87968832e+08],
       [6.53892544e+08, 6.53892544e+08, 5.27044448e+08, 5.27044448e+08],
       [8.27587200e+08, 8.27587200e+08, 7.28163136e+08, 7.28163136e+08],
       [1.44158864e+08, 1.44158864e+08, 1.30993048e

In [150]:
clf = LogisticRegression(random_state=0).fit(t1, tc)

In [151]:
clf.score(t1, tc)

0.43636363636363634

In [152]:
from sklearn.ensemble import ExtraTreesClassifier
selector = SelectFromModel(estimator=ExtraTreesClassifier(n_estimators=50)).fit(tf, tc)

In [153]:
t1=tf
selector.get_support()


array([False, False,  True,  True,  True,  True,  True,  True, False,
       False, False,  True, False,  True, False,  True,  True, False,
        True,  True,  True,  True,  True,  True, False,  True,  True,
       False, False, False,  True, False, False, False, False,  True,
        True, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
        True,  True, False, False,  True, False,  True, False,  True,
       False, False,  True, False,  True,  True, False, False, False,
        True,  True,  True, False, False,  True, False, False,  True,
        True, False, False, False, False,  True,  True])

In [154]:
t1=ef
selector.transform(t1)

array([[1.6774779e+03, 4.1133627e-01, 1.3995692e-02, ..., 5.6817696e+01,
        0.0000000e+00, 5.6550686e+01],
       [2.0706106e+03, 3.4537622e-01, 1.0768062e-02, ..., 1.5095553e+01,
        0.0000000e+00, 5.1756165e+01],
       [5.2834517e+03, 5.2794075e-01, 2.0350572e-02, ..., 1.1588735e+01,
        0.0000000e+00, 3.1967123e+01],
       ...,
       [2.0203104e+03, 3.4519389e-01, 1.0759536e-02, ..., 6.5421720e+00,
        1.0000000e+00, 6.5115067e+01],
       [9.4777429e+02, 2.8413337e-01, 8.0349334e-03, ..., 1.5923249e+01,
        1.0000000e+00, 6.0698631e+01],
       [1.1355579e+03, 4.5397761e-01, 1.6227460e-02, ..., 4.7212458e+00,
        1.0000000e+00, 6.0000000e+01]], dtype=float32)

In [155]:
clf = ExtraTreesClassifier(n_estimators=50).fit(t1, tc)

In [ ]:
clf.score(e1,ec)

array([[3.5559312e+03, 3.4892747e-01, 1.0934568e-02, ..., 1.8379179e+01,
        0.0000000e+00, 6.0413700e+01],
       [3.4340444e+03, 4.1096082e-01, 1.3976535e-02, ..., 1.6116812e+02,
        1.0000000e+00, 3.9000000e+01],
       [1.7294396e+03, 4.0873712e-01, 1.3863248e-02, ..., 9.7704079e+01,
        0.0000000e+00, 2.7904110e+01],
       ...,
       [3.3646284e+03, 4.8883533e-01, 1.8131876e-02, ..., 5.4801497e+00,
        0.0000000e+00, 6.0330395e+01],
       [2.1830483e+03, 2.6239780e-01, 7.1308143e-03, ..., 5.2007880e+00,
        0.0000000e+00, 6.0330395e+01],
       [1.9795342e+03, 4.7079375e-01, 1.7137401e-02, ..., 6.6197944e+00,
        0.0000000e+00, 5.6400002e+01]], dtype=float32)

1.0

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
t1=tf
e1=ef
tt1=ttf
print(e1.shape)
print(ec.shape)
clf = SVC(kernel='linear').fit(e1,ec)




torch.Size([36, 88])
torch.Size([36])


In [ ]:
t1=model.transform(t1)
e1=model.transform(e1)
tt1=model.transform(tt1)

In [ ]:
       """
            ground truth mask features: 
            
            """
            # t1ce computed mask shape features 
            shape=radiomics.shape.RadiomicsShape(t1ce,seg2)
            shape._initSegmentBasedCalculation()
            
            tfeatures[0]=shape.getMeshVolumeFeatureValue()
            tfeatures[1]=shape.getVoxelVolumeFeatureValue()
            tfeatures[2]=shape.getSurfaceAreaFeatureValue()
            if shape.SurfaceArea ==0:
                tfeatures[3]=0
            else:
                tfeatures[3]=shape.getSphericityFeatureValue()
            tfeatures[4]=shape.getCompactness1FeatureValue()
            tfeatures[5]=shape.getCompactness2FeatureValue()
            tfeatures[6]=shape.getSphericalDisproportionFeatureValue()
            tfeatures[7]=shape.getMaximum3DDiameterFeatureValue()
            tfeatures[8]=shape.getMaximum2DDiameterSliceFeatureValue()
            tfeatures[9]=shape.getMaximum2DDiameterColumnFeatureValue()
            tfeatures[10]=shape.getMaximum2DDiameterRowFeatureValue()
            tfeatures[11]=shape.getMajorAxisLengthFeatureValue()
            tfeatures[12]=shape.getMinorAxisLengthFeatureValue()
            tfeatures[13]=shape.getLeastAxisLengthFeatureValue()
            tfeatures[14]=shape.getElongationFeatureValue()
            tfeatures[15]=shape.getFlatnessFeatureValue()
            
            # flair computed mask shape features 
            shape=radiomics.shape.RadiomicsShape(flair,seg2)
            shape._initSegmentBasedCalculation()
            
            tfeatures[16]=shape.getMeshVolumeFeatureValue()
            tfeatures[17]=shape.getVoxelVolumeFeatureValue()
            tfeatures[18]=shape.getSurfaceAreaFeatureValue()
            if shape.SurfaceArea ==0:
                tfeatures[19]=0
                tfeatures[21]=0
                tfeatures[22]=0
            else:
                tfeatures[19]=shape.getSphericityFeatureValue()
                tfeatures[21]=shape.getCompactness2FeatureValue()
                tfeatures[22]=shape.getSphericalDisproportionFeatureValue()
                tfeatures[23]=shape.getMaximum3DDiameterFeatureValue()
                
            tfeatures[20]=shape.getCompactness1FeatureValue()
            tfeatures[24]=shape.getMaximum2DDiameterSliceFeatureValue()
            tfeatures[25]=shape.getMaximum2DDiameterColumnFeatureValue()
            tfeatures[26]=shape.getMaximum2DDiameterRowFeatureValue()
            tfeatures[27]=shape.getMajorAxisLengthFeatureValue()
            tfeatures[28]=shape.getMinorAxisLengthFeatureValue()
            tfeatures[29]=shape.getLeastAxisLengthFeatureValue()
            tfeatures[30]=shape.getElongationFeatureValue()
            tfeatures[31]=shape.getFlatnessFeatureValue()
            
            # t1ce and computed mask first order features
            fo = radiomics.firstorder.RadiomicsFirstOrder(t1ce,seg2)
            fo._initCalculation()

            tfeatures[32]=(torch.tensor(fo.getEnergyFeatureValue()))
            tfeatures[33]=(torch.tensor(fo.getTotalEnergyFeatureValue()))
            tfeatures[34]=(torch.tensor(fo.getEntropyFeatureValue()))
            tfeatures[35]=(torch.tensor(fo.getMinimumFeatureValue()))
            tfeatures[36]=(torch.tensor(fo.get10PercentileFeatureValue()))
            tfeatures[37]=(torch.tensor(fo.get90PercentileFeatureValue()))
            tfeatures[38]=(torch.tensor(fo.getMaximumFeatureValue()))
            tfeatures[39]=(torch.tensor(fo.getMeanFeatureValue()))
            tfeatures[40]=(torch.tensor(fo.getMedianFeatureValue()))
            tfeatures[41]=(torch.tensor(fo.getInterquartileRangeFeatureValue()))
            tfeatures[42]=(torch.tensor(fo.getRangeFeatureValue()))
            tfeatures[43]=(torch.tensor(fo.getMeanAbsoluteDeviationFeatureValue()))
            tfeatures[44]=(torch.tensor(fo.getRobustMeanAbsoluteDeviationFeatureValue()))
            tfeatures[45]=(torch.tensor(fo.getRootMeanSquaredFeatureValue()))
            tfeatures[46]=(torch.tensor(fo.getStandardDeviationFeatureValue()))
            tfeatures[47]=(torch.tensor(fo.getSkewnessFeatureValue()))
            tfeatures[48]=(torch.tensor(fo.getKurtosisFeatureValue()))
            tfeatures[49]=(torch.tensor(fo.getVarianceFeatureValue()))
            tfeatures[50]=(torch.tensor(fo.getUniformityFeatureValue()))
            
            # flair and computed mask first order features
            fo = radiomics.firstorder.RadiomicsFirstOrder(flair,seg2)
            fo._initCalculation()
            
            tfeatures[51]=(torch.tensor(fo.getEnergyFeatureValue()))
            tfeatures[52]=(torch.tensor(fo.getTotalEnergyFeatureValue()))
            tfeatures[53]=(torch.tensor(fo.getEntropyFeatureValue()))
            tfeatures[54]=(torch.tensor(fo.getMinimumFeatureValue()))
            tfeatures[55]=(torch.tensor(fo.get10PercentileFeatureValue()))
            tfeatures[56]=(torch.tensor(fo.get90PercentileFeatureValue()))
            tfeatures[57]=(torch.tensor(fo.getMaximumFeatureValue()))
            tfeatures[58]=(torch.tensor(fo.getMeanFeatureValue()))
            tfeatures[59]=(torch.tensor(fo.getMedianFeatureValue()))
            tfeatures[60]=(torch.tensor(fo.getInterquartileRangeFeatureValue()))
            tfeatures[61]=(torch.tensor(fo.getRangeFeatureValue()))
            tfeatures[62]=(torch.tensor(fo.getMeanAbsoluteDeviationFeatureValue()))
            tfeatures[63]=(torch.tensor(fo.getRobustMeanAbsoluteDeviationFeatureValue()))
            tfeatures[64]=(torch.tensor(fo.getRootMeanSquaredFeatureValue()))
            tfeatures[65]=(torch.tensor(fo.getStandardDeviationFeatureValue()))
            tfeatures[66]=(torch.tensor(fo.getSkewnessFeatureValue()))
            tfeatures[67]=(torch.tensor(fo.getKurtosisFeatureValue()))
            tfeatures[68]=(torch.tensor(fo.getVarianceFeatureValue()))
            tfeatures[69]=(torch.tensor(fo.getUniformityFeatureValue()))
            
            # t1ce and computed mask grey-level features
            grey = radiomics.glcm.RadiomicsGLCM(t1ce,seg2)
            grey._initCalculation()
            
            tfeatures[70]=(torch.tensor(grey.getClusterProminenceFeatureValue()))
            tfeatures[71]=(torch.tensor(grey.getClusterShadeFeatureValue()))
            tfeatures[72]=(torch.tensor(grey.getCorrelationFeatureValue()))
            
            # flair and computed mask grey-level features
            grey = radiomics.glcm.RadiomicsGLCM(flair,seg2)
            grey._initCalculation()
            
            tfeatures[73]=(torch.tensor(grey.getClusterProminenceFeatureValue()))
            tfeatures[74]=(torch.tensor(grey.getClusterShadeFeatureValue()))
            tfeatures[75]=(torch.tensor(grey.getCorrelationFeatureValue()))
            
            #t1ce grey-level zone features
            grey =radiomics.glszm.RadiomicsGLSZM(t1ce,seg2)
            grey._initCalculation()
            
            tfeatures[76]=(torch.tensor(grey.getLargeAreaEmphasisFeatureValue()))
            tfeatures[77]=(torch.tensor(grey.getGrayLevelNonUniformityFeatureValue()))
            tfeatures[78]=(torch.tensor(grey.getGrayLevelVarianceFeatureValue()))
            
            #flair grey-level zone features
            grey =radiomics.glszm.RadiomicsGLSZM(flair,seg2)
            grey._initCalculation()
            
            tfeatures[79]=(torch.tensor(grey.getLargeAreaEmphasisFeatureValue()))
            tfeatures[80]=(torch.tensor(grey.getGrayLevelNonUniformityFeatureValue()))
            tfeatures[81]=(torch.tensor(grey.getGrayLevelVarianceFeatureValue()))
            
            #t1ce grey-level dependence features
            grey =radiomics.gldm.RadiomicsGLDM(t1ce,seg2)
            grey._initCalculation()
            
            tfeatures[82]=(torch.tensor(grey.getSmallDependenceHighGrayLevelEmphasisFeatureValue()))
            tfeatures[83]=(torch.tensor(grey.getLargeDependenceLowGrayLevelEmphasisFeatureValue()))
            
            #t1ce grey-level dependence features
            grey =radiomics.gldm.RadiomicsGLDM(flair,seg2)
            grey._initCalculation()
            
            tfeatures[84]=(torch.tensor(grey.getSmallDependenceHighGrayLevelEmphasisFeatureValue()))
            tfeatures[85]=(torch.tensor(grey.getLargeDependenceLowGrayLevelEmphasisFeatureValue()))
            
            # clinical info
            if status == ['1']:
                tfeatures[86]=1
            else:
                tfeatures[86]=0
            tfeatures[87]=age
            train_tfeature[index]=tfeatures